# Advanced Operations in LangChain

This notebook covers the following operations:

- LLM Cost Monitoring
- Caching
- Streaming

## Install OpenAI, HuggingFace and LangChain dependencies

In [ ]:
!pip install langchain==0.1.19
!pip install langchain-openai==0.1.6
!pip install langchain-community==0.0.38

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Enter API Tokens

#### Enter your Open AI Key here

You can get the key from [here](https://platform.openai.com/api-keys) after creating an account or signing in

In [ ]:
from getpass import getpass

OPENAI_KEY = getpass('Please enter your Open AI API Key here: ')

Please enter your Open AI API Key here: ··········


## Setup necessary system environment variables

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY

## Chat Models and LLMs

Large Language Models (LLMs) are a core component of LangChain. LangChain does not implement or build its own LLMs. It provides a standard API for interacting with almost every LLM out there.

There are lots of LLM providers (OpenAI, Hugging Face, etc) - the LLM class is designed to provide a standard interface for all of them.

## Accessing Commercial LLMs like ChatGPT

In [ ]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

## Tracking LLM Costs

Typically LLMs like ChatGPT charge you based on the number of tokens per request and response. You can track your token usage for specific calls. It is currently only implemented for the OpenAI API.

In [ ]:
from langchain_community.callbacks import get_openai_callback

prompt = """Explain Generative AI in one line"""

with get_openai_callback() as cb:
    response = chatgpt.invoke(prompt)
    print(response.content)
    print(cb)

Generative AI is a type of artificial intelligence that can create new content, such as images, text, or music, based on patterns and data it has been trained on.
Tokens Used: 50
	Prompt Tokens: 15
	Completion Tokens: 35
Successful Requests: 1
Total Cost (USD): $9.25e-05


In [ ]:
cb.total_tokens

50

In [ ]:
cb.prompt_tokens, cb.completion_tokens

(15, 35)

In [ ]:
cb.total_cost

9.25e-05

## Caching in LangChain

LangChain includes an optional caching layer for language model APIs (LLMs). This caching feature is beneficial for two main reasons:

1. **Cost Efficiency:** By caching responses, you reduce the number of API calls made to LLM providers, especially helpful if you are frequently requesting the same completions. This can significantly lower costs.

2. **Performance Improvement:** Caching can enhance your application's speed by decreasing the need for repeated API calls to the LLM provider, making interactions quicker and more efficient.


#### InMemoryCache

In [ ]:
%%time
# integrations with other caching tools:
# https://api.python.langchain.com/en/latest/community_api_reference.html#module-langchain_community.cache
from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache

set_llm_cache(InMemoryCache())

# The first time, it is not yet in cache, so it should take longer

from langchain_core.prompts import ChatPromptTemplate

prompt = """Explain to me what is mortgage"""

chat_template = ChatPromptTemplate.from_template(prompt)

chatgpt.invoke(chat_template.format())

CPU times: user 323 ms, sys: 27.1 ms, total: 350 ms
Wall time: 2.1 s


AIMessage(content='A mortgage is a type of loan that is used to purchase a home or property. The borrower (homebuyer) agrees to pay back the loan amount, plus interest, over a set period of time. The property itself serves as collateral for the loan, meaning that if the borrower fails to make payments, the lender (usually a bank or financial institution) has the right to take possession of the property.', response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 15, 'total_tokens': 96}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ee67d2b5-1cb8-4139-ac68-3bd5a0df8a58-0')

In [ ]:
%%time
# The second time it is, so it goes faster
chatgpt.invoke(chat_template.format())

CPU times: user 2.47 ms, sys: 0 ns, total: 2.47 ms
Wall time: 4.33 ms


AIMessage(content='A mortgage is a type of loan that is used to purchase a home or property. The borrower (homebuyer) agrees to pay back the loan amount, plus interest, over a set period of time. The property itself serves as collateral for the loan, meaning that if the borrower fails to make payments, the lender (usually a bank or financial institution) has the right to take possession of the property.', response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 15, 'total_tokens': 96}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ee67d2b5-1cb8-4139-ac68-3bd5a0df8a58-0')

#### SQLite Cache

In [ ]:
# just to remove cache if it already exists, ignore if you get an error message below, that is normal (when cache doesnt exist)
!rm langchain.db

rm: cannot remove 'langchain.db': No such file or directory


In [ ]:
# We can do the same thing with a SQLite cache
from langchain.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path="langchain.db"))

In [ ]:
%%time

# The first time, it is not yet in cache, so it should take longer
prompt = """Explain to me what is fractional real estate"""

chat_template = ChatPromptTemplate.from_template(prompt)

chatgpt.invoke(chat_template.format())

CPU times: user 44.6 ms, sys: 2.89 ms, total: 47.5 ms
Wall time: 2.11 s


AIMessage(content='Fractional real estate is a type of investment where multiple investors come together to collectively own a share of a property. Each investor owns a fraction or percentage of the property, which can range from a few percentage points to a larger share. This allows individuals to invest in real estate without having to buy an entire property on their own. Fractional real estate investments can be in various types of properties such as residential homes, commercial buildings, or vacation properties. Investors can earn rental income and potential appreciation on their investment based on their ownership percentage.', response_metadata={'token_usage': {'completion_tokens': 107, 'prompt_tokens': 17, 'total_tokens': 124}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-36326e87-2243-4bf8-b40b-cb8612eca650-0')

In [ ]:
%%time
# The second time it is, so it goes faster
chatgpt.invoke(chat_template.format())

CPU times: user 82.1 ms, sys: 30.3 ms, total: 112 ms
Wall time: 113 ms


AIMessage(content='Fractional real estate is a type of investment where multiple investors come together to collectively own a share of a property. Each investor owns a fraction or percentage of the property, which can range from a few percentage points to a larger share. This allows individuals to invest in real estate without having to buy an entire property on their own. Fractional real estate investments can be in various types of properties such as residential homes, commercial buildings, or vacation properties. Investors can earn rental income and potential appreciation on their investment based on their ownership percentage.', response_metadata={'token_usage': {'completion_tokens': 107, 'prompt_tokens': 17, 'total_tokens': 124}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-36326e87-2243-4bf8-b40b-cb8612eca650-0')

## Streaming in LLMs

All language model interfaces (LLMs) in LangChain implement the `Runnable` interface, which provides default methods such as `ainvoke`, `batch`, `abatch`, `stream`, and `astream`. This setup equips all LLMs with basic streaming capabilities.

### Streaming Defaults:

- **Synchronous Streaming:** By default, streaming operations return an `Iterator` that yields a single value, the final result from the LLM provider.
- **Asynchronous Streaming:** Similarly, async streaming defaults to returning an `AsyncIterator` with the final result.

### Limitations:

- These default implementations do not support token-by-token streaming. For such detailed streaming, the LLM provider must offer native support. However, the default setup ensures that your code expecting an iterator of tokens will function correctly within these constraints.


In [ ]:
prompt = """Explain to me what is mortgage in detail with pros and cons"""
chat_template = ChatPromptTemplate.from_template(prompt)

for chunk in chatgpt.stream(chat_template.format()):
    print(chunk.content)


A
 mortgage
 is
 a
 type
 of
 loan
 that
 is
 used
 to
 purchase
 a
 home
 or
 other
 real
 estate
 property
.
 The
 borrower
 (
home
buyer
)
 agrees
 to
 pay
 back
 the
 loan
 amount
,
 plus
 interest
,
 over
 a
 specified
 period
 of
 time
.
 The
 property
 itself
 serves
 as
 collateral
 for
 the
 loan
,
 meaning
 that
 if
 the
 borrower
 fails
 to
 make
 their
 mortgage
 payments
,
 the
 lender
 has
 the
 right
 to
 fore
close
 on
 the
 property
 and
 sell
 it
 to
 rec
oup
 their
 losses
.


Pros
 of
 getting
 a
 mortgage
:

1
.
 Allows
 you
 to
 purchase
 a
 home
 without
 having
 to
 pay
 the
 full
 purchase
 price
 upfront
.

2
.
 Can
 help
 you
 build
 equity
 in
 your
 home
 over
 time
.

3
.
 Mortgage
 interest
 payments
 may
 be
 tax
-d
educt
ible
.

4
.
 Fixed
-rate
 mortgages
 offer
 predictable
 monthly
 payments
.


Cons
 of
 getting
 a
 mortgage
:

1
.
 You
 will
 be
 paying
 interest
 on
 the
 loan
 amount
,
 which
 can
 add
 up
 to
 a
 significant
 amount
 over
 the


In [ ]:
# Stream it as a contiguous block
prompt = """Explain to me what is mortgage in detail with pros and cons"""
chat_template = ChatPromptTemplate.from_template(prompt)

response = []
for chunk in chatgpt.stream(chat_template.format()):
    print(chunk.content, end="")
    response.append(chunk.content)

A mortgage is a type of loan that is used to purchase a home or real estate property. The borrower (homebuyer) agrees to pay back the loan amount, plus interest, over a specified period of time. The property itself serves as collateral for the loan, meaning that if the borrower fails to make their mortgage payments, the lender has the right to foreclose on the property and sell it to recoup their losses.

Pros of getting a mortgage:
1. Allows individuals to purchase a home without having to pay the full purchase price upfront.
2. Can help build equity in a property over time as the borrower pays down the loan.
3. Mortgage interest payments may be tax-deductible, providing potential tax benefits.
4. Fixed-rate mortgages offer predictable monthly payments, making budgeting easier.

Cons of getting a mortgage:
1. Borrowers may end up paying more in interest over the life of the loan compared to the original loan amount.
2. Defaulting on a mortgage can result in foreclosure and the loss of

In [ ]:
response[:10]

['', 'A', ' mortgage', ' is', ' a', ' type', ' of', ' loan', ' that', ' is']

In [ ]:
# Provides the same output as above
print(''.join(response))

A mortgage is a type of loan that is used to purchase a home or real estate property. The borrower (homebuyer) agrees to pay back the loan amount, plus interest, over a specified period of time. The property itself serves as collateral for the loan, meaning that if the borrower fails to make their mortgage payments, the lender has the right to foreclose on the property and sell it to recoup their losses.

Pros of getting a mortgage:
1. Allows individuals to purchase a home without having to pay the full purchase price upfront.
2. Can help build equity in a property over time as the borrower pays down the loan.
3. Mortgage interest payments may be tax-deductible, providing potential tax benefits.
4. Fixed-rate mortgages offer predictable monthly payments, making budgeting easier.

Cons of getting a mortgage:
1. Borrowers may end up paying more in interest over the life of the loan compared to the original loan amount.
2. Defaulting on a mortgage can result in foreclosure and the loss of